**Problem Statement**

To predict the probability that a person registering for a MedCamp will attend the camp.
[Hackathon Link](https://datahack.analyticsvidhya.com/contest/janatahack-healthcare-analytics/#ProblemStatement)


# Import Packages and Data

In [113]:
import pandas as pd
import numpy as np 
import os

# from pyforest import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
train_path= '/content/drive/My Drive/Healthcare/Train_2/Train/'

In [4]:
files = os.listdir(train_path)
files

['First_Health_Camp_Attended.csv',
 'Second_Health_Camp_Attended.csv',
 'Third_Health_Camp_Attended.csv',
 'Health_Camp_Detail.csv',
 'Train.csv',
 'Patient_Profile.csv',
 'Data_Dictionary.xlsx']

In [114]:
# Dictionary of all the files available
DataFrames = dict()

for file in files:
    if file.split(".")[1] == 'csv':
        name = 'df_'+file.split(".")[0]
        df_name = pd.read_csv(train_path + file)
        DataFrames[name] = df_name        
        
    else:
        name = 'df_'+file.split(".")[0]
        df_name = pd.read_excel(train_path + file)
        DataFrames[name] = df_name

In [115]:
print("Keys for the dictionary:: ")
DataFrames.keys()

Keys for the dictionary:: 


dict_keys(['df_First_Health_Camp_Attended', 'df_Second_Health_Camp_Attended', 'df_Third_Health_Camp_Attended', 'df_Health_Camp_Detail', 'df_Train', 'df_Patient_Profile', 'df_Data_Dictionary'])

In [116]:
DataFrames['df_Patient_Profile'].head()

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry
2,508307,0,0,0,0,3,87,46,02-Nov-02,D,BFSI
3,512612,0,0,0,0,1,75,47,02-Nov-02,D,Education
4,521075,0,0,0,0,3,None,80,24-Nov-02,H,Others


In [117]:
DataFrames['df_Health_Camp_Detail'].head()

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2
2,6544,03-Nov-03,15-Nov-03,First,F,1
3,6585,22-Nov-03,05-Dec-03,First,E,2
4,6561,30-Nov-03,18-Dec-03,First,E,1


In [173]:
test_df= pd.read_csv('/content/drive/My Drive/Healthcare/test_l0Auv8Q.csv')
test_df.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,505701,6548,21-May-06,1,0,0,0,2
1,500633,6584,02-Jun-06,0,0,0,0,0
2,506945,6582,10-Aug-06,0,0,0,0,0


# Pre processing

In [119]:
DataFrames['df_First_Health_Camp_Attended'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6218 entries, 0 to 6217
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Patient_ID      6218 non-null   int64  
 1   Health_Camp_ID  6218 non-null   int64  
 2   Donation        6218 non-null   int64  
 3   Health_Score    6218 non-null   float64
 4   Unnamed: 4      0 non-null      float64
dtypes: float64(2), int64(3)
memory usage: 243.0 KB


In [120]:
# Converting to Datetime format
DataFrames['df_Train'].Registration_Date = pd.to_datetime(DataFrames['df_Train'].Registration_Date)
DataFrames['df_Health_Camp_Detail'].Camp_Start_Date= pd.to_datetime(DataFrames['df_Health_Camp_Detail'].Camp_Start_Date)
DataFrames['df_Health_Camp_Detail'].Camp_End_Date= pd.to_datetime(DataFrames['df_Health_Camp_Detail'].Camp_End_Date)
DataFrames['df_Patient_Profile'].First_Interaction = pd.to_datetime(DataFrames['df_Patient_Profile'].First_Interaction)

In [121]:
DataFrames['df_Train']['Month'] = DataFrames['df_Train'].Registration_Date.dt.month
DataFrames['df_Train']['Year'] = DataFrames['df_Train'].Registration_Date.dt.year
DataFrames['df_Train']['DOW'] = DataFrames['df_Train'].Registration_Date.dt.dayofweek

In [122]:
DataFrames['df_Second_Health_Camp_Attended'].rename({"Health Score":"Health_Score"},axis = 1,inplace = True)

## Merging  Dataframes
For making a master file with patient information for analysis

In [123]:
df_master = DataFrames['df_Train'].merge(DataFrames['df_First_Health_Camp_Attended'],how = "left",on = ["Patient_ID",'Health_Camp_ID'])

In [124]:
df_master.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Donation,Health_Score,Unnamed: 4
0,489652,6578,2005-09-10,4,0,0,0,2,9.0,2005.0,5.0,NaN,NaN,NaN
1,507246,6578,2005-08-18,45,5,0,0,7,8.0,2005.0,3.0,NaN,NaN,NaN
2,523729,6534,2006-04-29,0,0,0,0,0,4.0,2006.0,5.0,NaN,NaN,NaN


In [125]:
df_master = df_master.merge(DataFrames['df_Second_Health_Camp_Attended'],how = "left",on = ["Patient_ID",'Health_Camp_ID'])

In [126]:
df_master.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Donation,Health_Score_x,Unnamed: 4,Health_Score_y
0,489652,6578,2005-09-10,4,0,0,0,2,9.0,2005.0,5.0,NaN,NaN,NaN,NaN
1,507246,6578,2005-08-18,45,5,0,0,7,8.0,2005.0,3.0,NaN,NaN,NaN,NaN
2,523729,6534,2006-04-29,0,0,0,0,0,4.0,2006.0,5.0,NaN,NaN,NaN,0.402054


In [127]:
DataFrames['df_Third_Health_Camp_Attended'].shape

(6515, 4)

In [128]:
df_master = df_master.merge(DataFrames['df_Third_Health_Camp_Attended'],how = "left",on = ["Patient_ID",'Health_Camp_ID'])
df_master.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Donation,Health_Score_x,Unnamed: 4,Health_Score_y,Number_of_stall_visited,Last_Stall_Visited_Number
0,489652,6578,2005-09-10,4,0,0,0,2,9.0,2005.0,5.0,NaN,NaN,NaN,NaN,2.0,1.0
1,507246,6578,2005-08-18,45,5,0,0,7,8.0,2005.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,523729,6534,2006-04-29,0,0,0,0,0,4.0,2006.0,5.0,NaN,NaN,NaN,0.402054,NaN,NaN


In [129]:
df_master.isnull().sum()

Patient_ID                       0
Health_Camp_ID                   0
Registration_Date              334
Var1                             0
Var2                             0
Var3                             0
Var4                             0
Var5                             0
Month                          334
Year                           334
DOW                            334
Donation                     69060
Health_Score_x               69060
Unnamed: 4                   75278
Health_Score_y               67459
Number_of_stall_visited      68763
Last_Stall_Visited_Number    68763
dtype: int64

In [130]:
# Dropping unnecessary columns
df_master.drop(["Donation","Unnamed: 4","Last_Stall_Visited_Number"],axis = 1, inplace= True)

In [131]:
df_master.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Health_Score_x,Health_Score_y,Number_of_stall_visited
0,489652,6578,2005-09-10,4,0,0,0,2,9.0,2005.0,5.0,NaN,NaN,2.0
1,507246,6578,2005-08-18,45,5,0,0,7,8.0,2005.0,3.0,NaN,NaN,NaN
2,523729,6534,2006-04-29,0,0,0,0,0,4.0,2006.0,5.0,NaN,0.402054,NaN


In [132]:
df_master['Target1'] = [0 if np.isnan(hs) else 1 for hs in df_master.Health_Score_x]
df_master['Target2'] = [0 if np.isnan(hs) else 1 for hs in df_master.Health_Score_y]
df_master['Target3'] = [0 if np.isnan(hs) else 1 for hs in df_master.Number_of_stall_visited]

df_master.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Health_Score_x,Health_Score_y,Number_of_stall_visited,Target1,Target2,Target3
0,489652,6578,2005-09-10,4,0,0,0,2,9.0,2005.0,5.0,NaN,NaN,2.0,0,0,1
1,507246,6578,2005-08-18,45,5,0,0,7,8.0,2005.0,3.0,NaN,NaN,NaN,0,0,0
2,523729,6534,2006-04-29,0,0,0,0,0,4.0,2006.0,5.0,NaN,0.402054,NaN,0,1,0


In [133]:
df_master['Target'] = df_master['Target1']+ df_master['Target2']+ df_master['Target3']

df_master.Target.value_counts()

0    54726
1    20552
Name: Target, dtype: int64

Hence there are patients who registered but did not show up to any of the camps.

In [32]:
DataFrames['df_Health_Camp_Detail'].head(3)

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,2003-08-16,2003-08-20,First,B,2
1,6530,2003-08-16,2003-10-28,First,C,2
2,6544,2003-11-03,2003-11-15,First,F,1


In [134]:
df_master = df_master.merge(DataFrames['df_Health_Camp_Detail'],how = "left",on = "Health_Camp_ID")

df_master.isnull().sum()

Patient_ID                     0
Health_Camp_ID                 0
Registration_Date            334
Var1                           0
Var2                           0
Var3                           0
Var4                           0
Var5                           0
Month                        334
Year                         334
DOW                          334
Health_Score_x             69060
Health_Score_y             67459
Number_of_stall_visited    68763
Target1                        0
Target2                        0
Target3                        0
Target                         0
Camp_Start_Date                0
Camp_End_Date                  0
Category1                      0
Category2                      0
Category3                      0
dtype: int64

In [145]:
df_master = df_master.merge(DataFrames['df_Patient_Profile'],how = "left",on = "Patient_ID")

df_master.isnull().sum()

Patient_ID                     0
Health_Camp_ID                 0
Registration_Date            334
Var1                           0
Var2                           0
Var3                           0
Var4                           0
Var5                           0
Month                        334
Year                         334
DOW                          334
Health_Score_x             69060
Health_Score_y             67459
Number_of_stall_visited    68763
Target1                        0
Target2                        0
Target3                        0
Target                         0
Camp_Start_Date                0
Camp_End_Date                  0
Category1                      0
Category2                      0
Category3                      0
Online_Follower                0
LinkedIn_Shared                0
Twitter_Shared                 0
Facebook_Shared                0
Income                         0
Education_Score                0
Age                            0
First_Inte

In [35]:
df_master.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Health_Score_x,Health_Score_y,Number_of_stall_visited,Target1,Target2,Target3,Target,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,489652,6578,2005-09-10,4,0,0,0,2,9.0,2005.0,5.0,NaN,NaN,2.0,0,0,1,1,2005-08-16,2005-10-14,Third,G,2,0,0,0,0,None,None,None,2004-12-06,NaN,NaN
1,507246,6578,2005-08-18,45,5,0,0,7,8.0,2005.0,3.0,NaN,NaN,NaN,0,0,0,0,2005-08-16,2005-10-14,Third,G,2,0,0,0,0,1,75,40,2004-09-08,C,Others
2,523729,6534,2006-04-29,0,0,0,0,0,4.0,2006.0,5.0,NaN,0.402054,NaN,0,1,0,1,2005-10-17,2007-11-07,Second,A,2,0,0,0,0,None,None,None,2004-06-22,NaN,NaN


In [36]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75278 entries, 0 to 75277
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Patient_ID               75278 non-null  int64         
 1   Health_Camp_ID           75278 non-null  int64         
 2   Registration_Date        74944 non-null  datetime64[ns]
 3   Var1                     75278 non-null  int64         
 4   Var2                     75278 non-null  int64         
 5   Var3                     75278 non-null  int64         
 6   Var4                     75278 non-null  int64         
 7   Var5                     75278 non-null  int64         
 8   Month                    74944 non-null  float64       
 9   Year                     74944 non-null  float64       
 10  DOW                      74944 non-null  float64       
 11  Health_Score_x           6218 non-null   float64       
 12  Health_Score_y           7819 no

In [146]:
df_master['FI_Month'] = df_master.First_Interaction.dt.month
df_master['FI_Year'] = df_master.First_Interaction.dt.year
df_master['FI_DOW'] = df_master.First_Interaction.dt.dayofweek

In [147]:
# keeping a copy before making changes
df_master_copy= df_master.copy(deep= True)

In [84]:
df_master.Category1.value_counts()

First     49892
Second    15114
Third     10272
Name: Category1, dtype: int64

In [ ]:
'''df_4_2.Category1.replace('First',0,inplace = True)
df_4_2.Category1.replace('Second',1,inplace = True)
df_4_2.Category1.replace('Third',2,inplace = True)

df_4_2.Category2.replace('A',0,inplace = True)
df_4_2.Category2.replace('B',1,inplace = True)
df_4_2.Category2.replace('C',2,inplace = True)
df_4_2.Category2.replace('D',3,inplace = True)
df_4_2.Category2.replace('E',4,inplace = True)
df_4_2.Category2.replace('F',5,inplace = True)
df_4_2.Category2.replace('G',6,inplace = True)

df_4_2.Category3.replace(1,0,inplace = True)
df_4_2.Category3.replace(2,1,inplace = True)'''

In [148]:
# Dropping columns
df_master.drop(["Health_Score_x","Health_Score_y","First_Interaction","City_Type","Employer_Category","Camp_Start_Date",
                       "Camp_End_Date","Number_of_stall_visited","Target1","Target2","Target3","Registration_Date","Patient_ID",
                       "Health_Camp_ID"],axis = 1, inplace= True)

In [86]:
df_master.isnull().sum()

Var1                 0
Var2                 0
Var3                 0
Var4                 0
Var5                 0
Month              334
Year               334
DOW                334
Target               0
Category1            0
Category2            0
Category3            0
Online_Follower      0
LinkedIn_Shared      0
Twitter_Shared       0
Facebook_Shared      0
Income               0
Education_Score      0
Age                  0
FI_Month             0
FI_Year              0
FI_DOW               0
dtype: int64

In [149]:
'''df_5.Month.fillna(-1,inplace = True)
df_5.Year.fillna(-1,inplace = True)
df_5.DOW.fillna(-1,inplace = True)'''

# Replacing missing values with garbage values
df_master.Month.fillna(13, inplace = True)
df_master.Year.fillna(0, inplace = True)
df_master.DOW.fillna(10, inplace = True)

In [55]:
df_master.head()

,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Target,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,FI_Month,FI_Year,FI_DOW
0,4,0,0,0,2,9.0,2005.0,5.0,1,2,6,1,0,0,0,0,None,None,None,12,2004,0
1,45,5,0,0,7,8.0,2005.0,3.0,0,2,6,1,0,0,0,0,1,75,40,9,2004,2
2,0,0,0,0,0,4.0,2006.0,5.0,1,1,0,1,0,0,0,0,None,None,None,6,2004,1
3,0,0,0,0,0,2.0,2004.0,5.0,0,0,4,1,0,0,0,0,None,None,None,2,2004,5
4,15,1,0,0,7,2.0,2006.0,1.0,1,1,0,1,0,0,0,1,1,70,40,7,2003,4


In [150]:
# to form a dataframe of values having low counts such that they can be grouped together

#lowval= pd.DataFrame(columns= ['Var1', 'Var2', 'Var3', 'Var4', 'Var5','Category1', 'Category2', 'Category3', 'Education_Score'])
lowval= pd.DataFrame(columns= ['Var1', 'Var2', 'Var5'])

for i in list(lowval.columns):
    x= pd.DataFrame(df_master[i].value_counts() )
    
    # to get all values below threshold
    s= pd.DataFrame(x[x.loc[:, i]< 20].index.sort_values(ascending= True))
    lowval.loc[:,i]= s[0]

lowval.shape

(106, 3)

In [151]:
lowval.tail()

,Var1,Var2,Var5
101,259,NaN,NaN
102,271,NaN,NaN
103,277,NaN,NaN
104,286,NaN,NaN
105,288,NaN,NaN


In [152]:
# Replacing to remove unnecessary levels
for col in list(lowval.columns):
  df_master[col].replace(to_replace= [lowval[col]], value= lowval[col][0], inplace= True)

df_master.Var1.value_counts() #for check

0     67445
1      2602
2      1383
3      1082
4       565
18      432
5       389
6       227
7       180
8       139
9       133
10      117
12      101
14       86
13       84
11       70
15       63
17       49
19       49
16       35
25       26
23       21
Name: Var1, dtype: int64

In [ ]:
df_master.Education_Score.value_counts()

None           65345
82               468
79               423
75               413
86               390
               ...  
92.07666667        1
71.05              1
70.2               1
68.56666667        1
74.45              1
Name: Education_Score, Length: 228, dtype: int64

In [153]:
# Replacing None(missing values) with unused values
df_master.Income.replace('None',7,inplace = True)
df_master.Education_Score.replace('None',0,inplace = True)
df_master.Age.replace('None',0,inplace = True)

In [154]:
# Creating bins for continuous features
df_master.Education_Score= df_master.Education_Score.astype(float)
df_master.Age= df_master.Age.astype(float)

df_master['EdScr_Bins']= pd.cut(df_master.Education_Score, bins= 5)
df_master['Age_Bins']= pd.cut(df_master.Age, bins= 7)

df_master.head()
#X['EdScr_Midpt']= X.EdScr_Bins.apply(lambda x: x.mid)

,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Target,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,FI_Month,FI_Year,FI_DOW,EdScr_Bins,Age_Bins
0,4,0,0,0,2,9.0,2005.0,5.0,1,Third,G,2,0,0,0,0,7,0.0,0.0,12,2004,0,"(-0.101, 20.2]","(-0.08, 11.429]"
1,18,5,0,0,7,8.0,2005.0,3.0,0,Third,G,2,0,0,0,0,1,75.0,40.0,9,2004,2,"(60.6, 80.8]","(34.286, 45.714]"
2,0,0,0,0,0,4.0,2006.0,5.0,1,Second,A,2,0,0,0,0,7,0.0,0.0,6,2004,1,"(-0.101, 20.2]","(-0.08, 11.429]"
3,0,0,0,0,0,2.0,2004.0,5.0,0,First,E,2,0,0,0,0,7,0.0,0.0,2,2004,5,"(-0.101, 20.2]","(-0.08, 11.429]"
4,15,1,0,0,7,2.0,2006.0,1.0,1,Second,A,2,0,0,0,1,1,70.0,40.0,7,2003,4,"(60.6, 80.8]","(34.286, 45.714]"


## Pre- processing for Test Data

In [174]:
test_df = test_df.merge(DataFrames['df_Health_Camp_Detail'],how = "left",on = "Health_Camp_ID")
test_df = test_df.merge(DataFrames['df_Patient_Profile'],how = "left",on = "Patient_ID")

#test_df= test_df2
test_df.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,505701,6548,21-May-06,1,0,0,0,2,2006-06-13,2006-08-18,Third,G,2,0,0,0,0,0,None,44,2003-02-05,E,NaN
1,500633,6584,02-Jun-06,0,0,0,0,0,2006-08-04,2006-08-09,Second,A,2,0,1,0,0,1,67,41,2004-12-11,D,Consulting
2,506945,6582,10-Aug-06,0,0,0,0,0,2006-04-06,2007-11-07,First,F,2,0,0,0,0,None,None,None,2006-04-19,NaN,NaN


In [175]:
test_df.First_Interaction = pd.to_datetime(test_df.First_Interaction)
test_df.Registration_Date =  pd.to_datetime(test_df.Registration_Date)

test_df['Month'] = test_df.Registration_Date.dt.month
test_df['Year'] = test_df.Registration_Date.dt.year
test_df['DOW'] = test_df.Registration_Date.dt.dayofweek

test_df['FI_Month'] = test_df.First_Interaction.dt.month
test_df['FI_Year'] = test_df.First_Interaction.dt.year
test_df['FI_DOW'] = test_df.First_Interaction.dt.dayofweek


test_df.Income.replace('None',7,inplace = True)
test_df.Education_Score.replace('None',0,inplace = True)
test_df.Age.replace('None',0,inplace = True)

test_df.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Month,Year,DOW,FI_Month,FI_Year,FI_DOW
0,505701,6548,2006-05-21,1,0,0,0,2,2006-06-13,2006-08-18,Third,G,2,0,0,0,0,0,0,44,2003-02-05,E,NaN,5,2006,6,2,2003,2
1,500633,6584,2006-06-02,0,0,0,0,0,2006-08-04,2006-08-09,Second,A,2,0,1,0,0,1,67,41,2004-12-11,D,Consulting,6,2006,4,12,2004,5
2,506945,6582,2006-08-10,0,0,0,0,0,2006-04-06,2007-11-07,First,F,2,0,0,0,0,7,0,0,2006-04-19,NaN,NaN,8,2006,3,4,2006,2


In [176]:
# to form a dataframe of value counts

#lowval= pd.DataFrame(columns= ['Var1', 'Var2', 'Var3', 'Var4', 'Var5','Category1', 'Category2', 'Category3', 'Education_Score'])
lowval2= pd.DataFrame(columns= ['Var1', 'Var2', 'Var5'])

for i in list(lowval2.columns):
    x= pd.DataFrame(test_df[i].value_counts() )
    
    # to get all values below threshold
    s= pd.DataFrame(x[x.loc[:, i]< 20].index.sort_values(ascending= True))
    lowval2.loc[:,i]= s[0]

lowval2.shape

(75, 3)

In [177]:
# Replacing to remove unnecessary levels
for col in list(lowval2.columns):
  test_df[col].replace(to_replace= [lowval2[col]], value= lowval2[col][0], inplace= True)


In [178]:
test_df.Education_Score= test_df.Education_Score.astype(float)
test_df.Age= test_df.Age.astype(float)

test_df['EdScr_Bins']= pd.cut(test_df.Education_Score, bins= 5)
test_df['Age_Bins']= pd.cut(test_df.Age, bins= 7)

test_df.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Month,Year,DOW,FI_Month,FI_Year,FI_DOW,EdScr_Bins,Age_Bins
0,505701,6548,2006-05-21,1,0,0,0,2,2006-06-13,2006-08-18,Third,G,2,0,0,0,0,0,0.0,44.0,2003-02-05,E,NaN,5,2006,6,2,2003,2,"(-0.097, 19.4]","(33.429, 44.571]"
1,500633,6584,2006-06-02,0,0,0,0,0,2006-08-04,2006-08-09,Second,A,2,0,1,0,0,1,67.0,41.0,2004-12-11,D,Consulting,6,2006,4,12,2004,5,"(58.2, 77.6]","(33.429, 44.571]"
2,506945,6582,2006-08-10,0,0,0,0,0,2006-04-06,2007-11-07,First,F,2,0,0,0,0,7,0.0,0.0,2006-04-19,NaN,NaN,8,2006,3,4,2006,2,"(-0.097, 19.4]","(-0.078, 11.143]"


## Label Encoding
For Categorical variables in both train and test data
the encoder needs to be fitted on both data of train and test and then each of the individual data variables need to be transformed

In [182]:
# Encoding for Education Score and Age
EdS= df_master.EdScr_Bins.append( test_df.EdScr_Bins)
AgeB= df_master.Age_Bins.append( test_df.Age_Bins)

In [183]:
le1= LabelEncoder()
le2= LabelEncoder()

le1.fit(EdS)
le2.fit(AgeB)

LabelEncoder()

In [184]:
df_master.EdScr_Bins= le1.transform(df_master.EdScr_Bins)
df_master.Age_Bins= le2.transform(df_master.Age_Bins)
df_master.head()

,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Target,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,FI_Month,FI_Year,FI_DOW,EdScr_Bins,Age_Bins
0,4,0,0,0,2,9.0,2005.0,5.0,1,Third,G,2,0,0,0,0,7,0.0,0.0,12,2004,0,0,0
1,18,5,0,0,7,8.0,2005.0,3.0,0,Third,G,2,0,0,0,0,1,75.0,40.0,9,2004,2,5,5
2,0,0,0,0,0,4.0,2006.0,5.0,1,Second,A,2,0,0,0,0,7,0.0,0.0,6,2004,1,0,0
3,0,0,0,0,0,2.0,2004.0,5.0,0,First,E,2,0,0,0,0,7,0.0,0.0,2,2004,5,0,0
4,15,1,0,0,7,2.0,2006.0,1.0,1,Second,A,2,0,0,0,1,1,70.0,40.0,7,2003,4,5,5


In [185]:
test_df.EdScr_Bins= le1.transform(test_df.EdScr_Bins)
test_df.Age_Bins= le2.transform(test_df.Age_Bins)
test_df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Month,Year,DOW,FI_Month,FI_Year,FI_DOW,EdScr_Bins,Age_Bins
0,505701,6548,2006-05-21,1,0,0,0,2,2006-06-13,2006-08-18,Third,G,2,0,0,0,0,0,0.0,44.0,2003-02-05,E,NaN,5,2006,6,2,2003,2,1,4
1,500633,6584,2006-06-02,0,0,0,0,0,2006-08-04,2006-08-09,Second,A,2,0,1,0,0,1,67.0,41.0,2004-12-11,D,Consulting,6,2006,4,12,2004,5,4,4
2,506945,6582,2006-08-10,0,0,0,0,0,2006-04-06,2007-11-07,First,F,2,0,0,0,0,7,0.0,0.0,2006-04-19,NaN,NaN,8,2006,3,4,2006,2,1,1
3,497447,6551,2006-08-27,0,0,0,0,0,2006-11-13,2006-11-18,Second,D,2,0,0,0,0,0,0.0,47.0,2006-08-25,B,NaN,8,2006,6,8,2006,4,1,6
4,496446,6533,2006-09-19,0,0,0,0,0,2006-09-20,2006-09-23,First,E,2,0,0,0,0,7,0.0,0.0,2006-09-19,B,NaN,9,2006,1,9,2006,1,1,1


In [186]:
df_master.drop(["Education_Score", "Age"], axis= 1, inplace= True)
#X.EdScr_Bins.value_counts()

In [187]:
# Encoding for Categories
Cat1= df_master.Category1.append(test_df.Category1)
Cat2= df_master.Category2.append(test_df.Category2)
Cat3= df_master.Category3.append(test_df.Category3)

In [188]:
le_Cat1= LabelEncoder()
le_Cat1.fit(Cat1)

le_Cat2= LabelEncoder()
le_Cat2.fit(Cat2)

le_Cat3= LabelEncoder()
le_Cat3.fit(Cat3)

LabelEncoder()

In [189]:
df_master.Category1= le_Cat1.transform(df_master.Category1)
#print(df_master.Category1.value_counts() )

df_master.Category2= le_Cat2.transform(df_master.Category2)
#print(df_master.Category2.value_counts())

df_master.Category3= le_Cat3.transform(df_master.Category3)
#print(df_master.Category3.value_counts() )

test_df.Category1= le_Cat1.transform(test_df.Category1)
test_df.Category2= le_Cat2.transform(test_df.Category2)
test_df.Category3= le_Cat3.transform(test_df.Category3)

In [190]:
df_master.head(3)

,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Target,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,FI_Month,FI_Year,FI_DOW,EdScr_Bins,Age_Bins
0,4,0,0,0,2,9.0,2005.0,5.0,1,2,6,1,0,0,0,0,7,12,2004,0,0,0
1,18,5,0,0,7,8.0,2005.0,3.0,0,2,6,1,0,0,0,0,1,9,2004,2,5,5
2,0,0,0,0,0,4.0,2006.0,5.0,1,1,0,1,0,0,0,0,7,6,2004,1,0,0


# Model Fitting

In [191]:
X = df_master.drop("Target",axis = 1)
Y = df_master['Target']

## Correlation Check of Predictors

In [192]:
# Theil's U for correlation between categorical variables
# Ref: https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9

from collections import Counter
from scipy.stats import entropy
import math

def conditional_entropy(x,y):
    # entropy of x given y
    y_counter = Counter(y)
    xy_counter = Counter(list(zip(x,y)))
    total_occurrences = sum(y_counter.values())
    entropy = 0
    for xy in xy_counter.keys():
        p_xy = xy_counter[xy] / total_occurrences
        p_y = y_counter[xy[1]] / total_occurrences
        entropy += p_xy * math.log(p_y/p_xy)
    return entropy

def theil_u(x,y):
    s_xy = conditional_entropy(x,y)
    x_counter = Counter(x)
    total_occurrences = sum(x_counter.values())
    p_x = list(map(lambda n: n/total_occurrences, x_counter.values()))
    s_x = entropy(p_x)
    if s_x == 0:
        return 1
    else:
        return (s_x - s_xy) / s_x    

In [193]:
theilu = pd.DataFrame(index=X.columns,columns= X.columns)
columns = list(theilu.columns)
for row in columns:   
    for col in columns:
        u = theil_u(X[row].tolist(),X[col].tolist())
        theilu.loc[row, col] = u
theilu.fillna(value=np.nan,inplace=True)

In [194]:
theilu.style.background_gradient(cmap= 'Blues')

,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,FI_Month,FI_Year,FI_DOW,EdScr_Bins,Age_Bins
Var1,1.000000,0.195136,0.003559,0.072711,0.462660,0.010939,0.016881,0.004155,0.001166,0.011287,0.001078,0.016016,0.022135,0.013146,0.012993,0.058393,0.055886,0.198027,0.012338,0.031862,0.056402
Var2,0.513128,1.000000,0.014874,0.118892,0.526779,0.029747,0.023407,0.016447,0.005539,0.022242,0.001796,0.026417,0.035158,0.018604,0.015285,0.079161,0.061060,0.170576,0.036543,0.038374,0.076292
Var3,0.586209,0.931667,1.000000,0.502554,0.939456,0.039990,0.016049,0.008504,0.000563,0.014429,0.000612,0.005261,0.007438,0.005744,0.005270,0.238949,0.273144,0.521527,0.179853,0.008723,0.438162
Var4,0.442397,0.275091,0.018564,1.000000,0.499313,0.007203,0.007920,0.003568,0.002453,0.010998,0.000388,0.011583,0.013759,0.010605,0.009863,0.054143,0.050180,0.268754,0.011768,0.028861,0.048436
Var5,0.604828,0.261886,0.007456,0.107283,1.000000,0.012838,0.018121,0.004623,0.001612,0.013274,0.001183,0.017276,0.025331,0.013645,0.013844,0.064665,0.048766,0.193294,0.015010,0.035035,0.063025
Month,0.002480,0.002565,0.000055,0.000268,0.002227,1.000000,0.129799,0.017536,0.045371,0.096966,0.002584,0.000409,0.000505,0.000386,0.000454,0.004129,0.178879,0.040404,0.006575,0.006163,0.003445
Year,0.007693,0.004057,0.000044,0.000593,0.006317,0.260914,1.000000,0.026221,0.136413,0.278494,0.009959,0.001994,0.002687,0.001845,0.001985,0.014548,0.060736,0.383681,0.003712,0.013603,0.012125
DOW,0.001190,0.001791,0.000015,0.000168,0.001013,0.022153,0.016478,1.000000,0.001101,0.006420,0.001413,0.000048,0.000034,0.000071,0.000030,0.000475,0.002350,0.001366,0.021476,0.000093,0.000497
Category1,0.000757,0.001367,0.000002,0.000262,0.000800,0.129822,0.194177,0.002493,1.000000,1.000000,0.002574,0.000298,0.000334,0.000215,0.000294,0.001728,0.029504,0.063118,0.001150,0.000382,0.001123
Category2,0.003884,0.002910,0.000030,0.000622,0.003494,0.147168,0.210273,0.007714,0.530424,1.000000,0.002007,0.000714,0.001001,0.000613,0.000691,0.011121,0.031643,0.091029,0.003322,0.012832,0.010101


In [511]:
################ If dropping variable after feature importance check,  then only run this code #########
X_org= X.copy(deep= True)

#X.drop(["Var3", "Var4", "Online_Follower", "LinkedIn_Shared", "Twitter_Shared", "Facebook_Shared"], axis= 1, inplace= True)
X.drop(["Var3", "Var4", "Online_Follower", "Twitter_Shared", "Facebook_Shared"], axis= 1, inplace= True)
X.head(3)

,Var1,Var2,Var5,Month,Year,DOW,Category1,Category2,Category3,LinkedIn_Shared,Income,FI_Month,FI_Year,FI_DOW,EdScr_Bins,Age_Bins
0,4,0,2,9.0,2005.0,5.0,2,6,1,0,7,12,2004,0,0,0
1,45,5,7,8.0,2005.0,3.0,2,6,1,0,1,9,2004,2,5,5
2,0,0,0,4.0,2006.0,5.0,1,0,1,0,7,6,2004,1,0,0


In [195]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0,stratify = Y) 

In [196]:
# describes info about train and test set 
print("Number transactions X_train dataset: ", X_train.shape) 
print("Number transactions y_train dataset: ", y_train.shape) 
print("Number transactions X_test dataset: ", X_test.shape) 
print("Number transactions y_test dataset: ", y_test.shape) 

Number transactions X_train dataset:  (52694, 21)
Number transactions y_train dataset:  (52694,)
Number transactions X_test dataset:  (22584, 21)
Number transactions y_test dataset:  (22584,)


In [197]:
def Training_Report(Model,x_train= X_train,x_test= X_test,Y_train= y_train,Y_test= y_test,label = 'C'):
    
    Model.fit(x_train, Y_train.ravel()) 
    predictions = Model.predict(x_test)
    
    print('\n',label)

    # print classification report 
    print('\n Classification Report:\n',classification_report(Y_test, predictions)) 
    
    #Confusion Matrix
    conf_mat = confusion_matrix(y_true=Y_test, y_pred=predictions)

    print('\n Confusion matrix:\n', conf_mat)
    

    print('\n ROC_AUC score:\n',roc_auc_score(Y_test,predictions))

In [198]:
#Decision Tree
DT = DecisionTreeClassifier(random_state = 0) 

#Random Forest
RF = RandomForestClassifier(random_state = 0,n_jobs = -1)

# Gradient Boost
GB = GradientBoostingClassifier(learning_rate=0.01,random_state=0,n_estimators=1000)

In [199]:
Training_Report(GB,label = 'Gradient Boosting')


 Gradient Boosting

 Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.88      0.86     16418
           1       0.65      0.59      0.61      6166

    accuracy                           0.80     22584
   macro avg       0.75      0.73      0.74     22584
weighted avg       0.79      0.80      0.80     22584


 Confusion matrix:
 [[14435  1983]
 [ 2555  3611]]

 ROC_AUC score:
 0.7324244052762513


In [200]:
# Gradient Boost
#GB = GradientBoostingClassifier(learning_rate=0.01,random_state=0,n_estimators=1000)
GB.fit(X,Y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=1000,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [201]:
# To check for feature importance
GB_feat= pd.DataFrame(GB.feature_importances_).T
GB_feat.columns= X.columns
#GB_feat.plot(kind= 'bar', figsize= (10, 15))
GB_feat

,Var1,Var2,Var3,Var4,Var5,Month,Year,DOW,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,FI_Month,FI_Year,FI_DOW,EdScr_Bins,Age_Bins
0,0.008303,0.000896,0.0,0.00006,0.00754,0.051262,0.04345,0.00114,0.738059,0.06982,0.001485,0.000477,0.001312,0.000608,0.000048,0.018499,0.008803,0.007655,0.002251,0.006087,0.032247


In [517]:
# Dropping features and checking
GB_feat= pd.DataFrame(GB.feature_importances_).T
GB_feat.columns= X.columns
#GB_feat.plot(kind= 'bar', figsize= (10, 15))
GB_feat

,Var1,Var2,Var5,Month,Year,DOW,Category1,Category2,Category3,LinkedIn_Shared,Income,FI_Month,FI_Year,FI_DOW,EdScr_Bins,Age_Bins
0,0.008457,0.000858,0.00834,0.05203,0.039972,0.004068,0.740683,0.067002,0.001488,0.001419,0.018515,0.008784,0.007922,0.002405,0.005516,0.032541


# Prediction

In [203]:
pred = GB.predict_proba(test_df[X.columns])

In [204]:
test_df['Outcome'] = [g[1] for g in pred]

In [205]:
df_Submission = test_df[['Patient_ID','Health_Camp_ID',"Outcome"]]

In [206]:
df_Submission.to_csv('/content/drive/My Drive/Healthcare/Submission_1_team_Loop.csv',index = False)

In [207]:
df_Submission.head()

,Patient_ID,Health_Camp_ID,Outcome
0,505701,6548,0.701204
1,500633,6584,0.581208
2,506945,6582,0.098020
3,497447,6551,0.794684
4,496446,6533,0.074745
